In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import requests

response = requests.get('https://www.hostinger.com/tutorials/how-to-run-a-python-script-in-linux')

print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>


<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', f

In [3]:
import os

In [4]:
output_dir = 'images/lavender'
os.makedirs(output_dir, exist_ok=True)

In [7]:
URL = "https://www.pexels.com/search/lavender%20flower/"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

with requests.Session() as session:
    response = session.get(URL, headers=HEADERS)
    if response.status_code != 200:
        print(f"Failed to fetch the page: {response.status_code}")
    else:
        soup = BeautifulSoup(response.content, 'html.parser')

        images = soup.find_all('img')

        if not images:
            print("No images found — site might be blocking scrapers.")

        for i, img in enumerate(images):
            src = img.get('src')
            if src and src.startswith('http'):
                try:
                    img_data = session.get(src).content
                    with open(f'{output_dir}/lavender_{i}.jpg', 'wb') as f:
                        f.write(img_data)
                    print(f"Saved: lavender_{i}.jpg")
                except Exception as e:
                    print(f"Failed to download {src}: {e}")

Failed to fetch the page: 403
